In [1]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    GenerationConfig
)
from tqdm import tqdm
from trl import SFTTrainer
import time
import pandas as pd
import numpy as np
from huggingface_hub import interpreter_login

interpreter_login()

/Users/pat/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/pat/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/pat/Library/Python/3.9/lib/python/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token is valid (permission: read).
Your token has been saved

In [5]:
import os
import glob
import shutil
os.environ['WANDB_DISABLED']="true"

# AI Related installing Packages

In [44]:
import torch
import zipfile
import torchaudio
import matplotlib
import timm
import librosa

# Speech to Text

In [29]:
cv_16 = load_dataset("mozilla-foundation/common_voice_16_1", "th", use_auth_token=True)

/Users/pat/Library/Python/3.9/lib/python/site-packages/datasets/load.py:2516: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
/Users/pat/Library/Python/3.9/lib/python/site-packages/datasets/load.py:1461: FutureWarning: The repository for mozilla-foundation/common_voice_16_1 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_16_1
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Reading metadata...: 32789it [00:00, 258101.93it/s]es/s]
Generating train split: 32789 examples [00:03, 9520.38 examples/s] 
Reading metadata...: 11038it [00:00, 292632.03i

In [32]:
cv_16

DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 32789
    })
    validation: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 11038
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 11038
    })
    other: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 205532
    })
    invalidated: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num

In [59]:
train  =  cv_16['train'].remove_columns(['accent','variant','segment','gender','age','client_id','locale'])
val = cv_16['validation'].remove_columns(['accent','variant','segment','gender','age','client_id','locale'])
test = cv_16['test'].remove_columns(['accent','variant','segment','gender','age','client_id','locale'])


In [86]:
audio = librosa.resample(y=audio,orig_sr=48000,target_sr=16000)

In [103]:
spec = librosa.stft(audio,n_fft=2048)

In [107]:
spec

array([[ 1.40180641e-08+0.00000000e+00j,  1.43452088e-04+0.00000000e+00j,
         2.98102860e-04+0.00000000e+00j, ...,
        -2.71480013e-05+0.00000000e+00j, -1.17502618e-05+0.00000000e+00j,
         8.91649526e-05+0.00000000e+00j],
       [ 8.91292279e-09+1.88533876e-08j,  5.91775825e-06+1.84526781e-04j,
        -1.22422404e-04-6.08518032e-05j, ...,
         3.48718885e-04-8.63319309e-05j, -1.02888720e-04+1.10638676e-04j,
        -7.39410977e-05-2.67159378e-05j],
       [-2.14371439e-08+3.04601677e-08j, -2.52293741e-04+1.08765661e-04j,
         5.26982345e-04+4.28556743e-04j, ...,
        -5.40024598e-04-4.44289330e-04j,  1.08508855e-05+1.41702838e-04j,
         1.02324181e-04+2.27756802e-05j],
       ...,
       [ 1.18098005e-11-1.13526442e-12j, -2.63099498e-08+8.63649019e-10j,
        -1.37562793e-08-1.07847533e-08j, ...,
         1.68813312e-08-8.44297726e-09j,  2.02497398e-09+1.39816827e-08j,
         1.87490501e-08+2.22647932e-09j],
       [ 1.14327213e-11-5.76775382e-13j, -2.

In [106]:
torch.nn.functional.normalize(spec)

AttributeError: 'numpy.ndarray' object has no attribute 'norm'